In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

train_data = pd.read_csv(r"D:\Projects\clickstream_data\train_data.csv")
test_data = pd.read_csv(r"D:\Projects\clickstream_data\test_data.csv")

In [3]:
train_data

year  month  day  order  country  session_id  page1_main_category  \
0       2008      6   22     21       29       15648                    3   
1       2008      5   19      6       29       10018                    2   
2       2008      7   15      2       29       19388                    3   
3       2008      5    2      2       29        7181                    2   
4       2008      6    9     16       29       13493                    2   
...      ...    ...  ...    ...      ...         ...                  ...   
132374  2008      7    4      3       29       17622                    4   
132375  2008      6   19      9       29       15165                    3   
132376  2008      7   15      4       29       19359                    1   
132377  2008      7   28     16       29       21454                    3   
132378  2008      7    6     17       29       17897                    2   

       page2_clothing_model  colour  location  model_photography  price  \
0                       C20      13         1                  2     48   
1                       B26      13         3                  1     57   
2                       C13       9         5                  1     48   
3                       B11       2         4                  1     43   
4                       B31       9         5                  1     57   
...                     ...     ...       ...                ...    ...   
132374                  P19       2         1                  1     48   
132375                  C26      14         3                  1     28   
132376                   A4       3         2                  2     38   
132377                  C50       9         5                  2     20   
132378                   B1       4         1                  1     57   

        price_2  page  
0             1     2  
1             1     2  
2             1     1  
3             2     1  
4             1     2  
...         ...   ...  
132374        1     2  
132375        2     2  
132376        2     1  
132377        2     3  
132378        1     1  

[132379 rows x 14 columns]

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132379 entries, 0 to 132378
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  132379 non-null  int64 
 1   month                 132379 non-null  int64 
 2   day                   132379 non-null  int64 
 3   order                 132379 non-null  int64 
 4   country               132379 non-null  int64 
 5   session_id            132379 non-null  int64 
 6   page1_main_category   132379 non-null  int64 
 7   page2_clothing_model  132379 non-null  object
 8   colour                132379 non-null  int64 
 9   location              132379 non-null  int64 
 10  model_photography     132379 non-null  int64 
 11  price                 132379 non-null  int64 
 12  price_2               132379 non-null  int64 
 13  page                  132379 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 14.1+ MB


LabelEncoder for catogorical features 

In [5]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
train_data['page2_clothing_model'] = le1.fit_transform(train_data['page2_clothing_model'])

le2 = LabelEncoder()
test_data['page2_clothing_model'] = le2.fit_transform(test_data['page2_clothing_model'])

StandardScaler

In [6]:
from sklearn.preprocessing import StandardScaler
train_features = train_data[['page1_main_category', 'page2_clothing_model', 'colour']]
train_target = train_data['price']

test_features = test_data[['page1_main_category', 'page2_clothing_model', 'colour']]
test_target = test_data['price']

scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)


In [7]:
train_features.shape

(132379, 3)

Model Evaluation and Hyperparameter Tuning


In [8]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [9]:
model_params = {
    "Linear Regressor": (LinearRegression(), {}),
    
    "Ridge Regressor": (Ridge(), {
        "alpha": [0.01, 0.1, 1, 10, 100]
    }),
    
    "Lasso Regressor": (Lasso(), {
        "alpha": [0.01, 0.1, 1, 10, 100]
    }),
    
    "Gradient Boosting Regressor": (GradientBoostingRegressor(), {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7]
    }),
    
    "Random Forest Regressor": (RandomForestRegressor(), {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    })
}

In [12]:
reports = []

for name, (model, param_grid) in model_params.items():
    if param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring="r2", n_jobs=-1)
        grid_search.fit(train_features, train_target)
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        best_model = model
        best_model.fit(train_features, train_target)
        best_params = "Default Parameters"

    predictions = best_model.predict(test_features)
    mae = mean_absolute_error(test_target, predictions)
    r2 = r2_score(test_target, predictions)

    reports.append((name, best_model, best_params, mae, r2))

In [15]:
for name, model, best_params, mae, r2 in reports:
    print(f"Model: {name}")
    print(f"Best Parameters: {best_params}")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("\n")

Model: Linear Regressor
Best Parameters: Default Parameters
MAE: 10.0456
R2 Score: 0.1364


Model: Ridge Regressor
Best Parameters: {'alpha': 10}
MAE: 10.0454
R2 Score: 0.1364


Model: Lasso Regressor
Best Parameters: {'alpha': 0.01}
MAE: 10.0402
R2 Score: 0.1364


Model: Gradient Boosting Regressor
Best Parameters: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300}
MAE: 0.4849
R2 Score: 0.9573


Model: Random Forest Regressor
Best Parameters: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 100}
MAE: 0.4429
R2 Score: 0.9547




In [16]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Price Prediction (Regression)")

for name, model, best_params, mae, r2 in reports:
    with mlflow.start_run(run_name=name) as run:
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("R2", r2)
        
        if name == "Linear Regressor":
            mlflow.sklearn.log_model(model, "linear_model")
        elif name == "Ridge Regressor":
            mlflow.sklearn.log_model(model, "ridge_model")
        elif name == "Lasso Regressor":
            mlflow.sklearn.log_model(model, "lasso_model")
        elif name == "Gradient Boosting Regressor":
            mlflow.sklearn.log_model(model, "gradient_boosting_model")
        elif name == "Random Forest Regressor":
            mlflow.sklearn.log_model(model, "random_forest_model")
        else:
            pass

In [ ]:
model_name ='Gradient Boosting Regressor'
run_id = '911e85dc72c64be89829b853abf339b6'
model_uri = f'runs:/{run_id}/gradient_boosting_model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri= model_uri , name= model_name)

Successfully registered model 'Gradient Boosting Regressor'.
2025/02/18 10:34:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Gradient Boosting Regressor, version 1


🏃 View run Gradient Boosting Regressor at: http://127.0.0.1:5000/#/experiments/332772607796571085/runs/911e85dc72c64be89829b853abf339b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/332772607796571085


Created version '1' of model 'Gradient Boosting Regressor'.


In [18]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_name = "Gradient Boosting Regressor"
model_version = "1"
model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.pyfunc.load_model(model_uri)

c:\Users\abina\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pickle

with open('le1_clothing_model.pkl', 'wb') as f:
    pickle.dump(le1, f)

with open('le2_clothing_model.pkl', 'wb') as f:
    pickle.dump(le2, f)

with open('regression_standard_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('gradient_boosting_model.pkl', 'wb') as f:
    pickle.dump(model, f)